In [ ]:
%pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from urllib.parse import urlparse, urljoin
from collections import deque
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import pandas as pd
import time




# API key - replace with your actual API key
GOOGLE_MAPS_API_KEY = 'enter your key'
# APIs and search parameters
TEXT_SEARCH_URL = "https://maps.googleapis.com/maps/api/place/textsearch/json"
PLACE_DETAILS_URL = "https://maps.googleapis.com/maps/api/place/details/json"
SEARCH_TAGS = ["pet grooming"]  # Add more tags as needed
SOCIAL_PLATFORMS = ["facebook.com", "instagram.com", "twitter.com", "linkedin.com"]

# User agents for web scraping
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
]

# ----------------- GOOGLE PLACES API FUNCTIONS -----------------

def search_places(query, location="Los Angeles"):
    """
    Search for places using Google Places API
    """
    print(f"\n🔍 Searching for: {query} in {location}")
    params = {"query": f"{query} in {location}", "key": GOOGLE_MAPS_API_KEY}
    response = requests.get(TEXT_SEARCH_URL, params=params)

    if response.status_code != 200:
        print(f"❌ API Error: {response.status_code}, {response.text}")
        return []

    results = response.json().get("results", [])
    place_ids = [r.get("place_id") for r in results]
    print(f"✅ Found {len(place_ids)} places")
    return place_ids

def get_place_details(place_id):
    """
    Get detailed information about a place using its place_id
    """
    params = {
        "place_id": place_id,
        "fields": "name,formatted_address,website,url,formatted_phone_number",
        "key": GOOGLE_MAPS_API_KEY
    }
    response = requests.get(PLACE_DETAILS_URL, params=params)

    if response.status_code != 200:
        print(f"❌ API Error: {response.status_code}, {response.text}")
        return {}

    return response.json().get("result", {})

# ----------------- WEB SCRAPING FUNCTIONS -----------------

def setup_chrome_driver():
    """
    Setup headless Chrome driver for web scraping
    """
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.binary_location = "/usr/bin/google-chrome"  # For Colab
    options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")
    return webdriver.Chrome(options=options)


    #return webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=options)
def extract_links_and_content(url):
    """
    Extract links and content from a webpage
    """
    try:
        headers = {"User-Agent": random.choice(USER_AGENTS)}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract text content
        all_text = soup.get_text(separator=" ", strip=True)

        # Extract links
        links = [a['href'] for a in soup.find_all('a', href=True)]

        return links, all_text
    except Exception as e:
        print(f"⚠️ Scrape error for {url}: {e}")
        return [], ""

def is_internal_link(link, domain):
    """
    Check if a link is internal to the domain
    """
    return domain in link and not any(bad in link for bad in ["indeed", "glassdoor", "ziprecruiter", "monster", "linkedin"])

def find_social_links(links):
    """
    Find social media links in a list of links
    """
    social_links = []
    for link in links:
        if any(social in link for social in SOCIAL_PLATFORMS):
            social_links.append(link)
    return social_links

def get_social_from_google(place_result):
    """
    Try to extract social links from Google Places result
    """
    social_links = []
    for field in ['url', 'website']:
        if field in place_result and place_result[field]:
            url = place_result[field]
            if any(social in url for social in SOCIAL_PLATFORMS):
                social_links.append(url)
    return social_links

In [ ]:
def extract_emails(text):
    """
    Extract valid email addresses from text
    """
    pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    emails = re.findall(pattern, text)

    # Filter for business/hiring related emails
    valid_emails = []
    for email in emails:
        email = email.lower()
        # Include hiring-related and general contact emails
        if any(email.startswith(prefix) for prefix in ["hr@", "jobs@", "careers@", "hiring@", "recruiting@", "info@", "contact@", "hello@"]):
            valid_emails.append(email)
        # Include Gmail accounts (often used by small businesses)
        elif email.endswith("@gmail.com"):
            valid_emails.append(email)

    return list(set(valid_emails))

def detect_job_content(text):
    """
    Detect job-related content in text
    """
    job_keywords = [
        "we're hiring", "now hiring", "join our team", "career", "careers", "job opening",
        "position available", "apply now", "job application", "employment", "work with us",
        "job opportunity", "open position", "join us"
    ]

    text_lower = text.lower()

    for keyword in job_keywords:
        if keyword in text_lower:
            # Find the context around the keyword
            start = max(0, text_lower.find(keyword) - 100)
            end = min(len(text_lower), text_lower.find(keyword) + 100)
            context = text[start:end].strip()
            return True, context

    return False, ""

def crawl_site(site_url, max_pages=5):
    """
    Crawl a website to find job-related content and contact information
    """
    print(f"\n🔍 Crawling: {site_url}")

    # Setup
    visited = set()
    queue = deque([site_url])
    emails = set()
    job_content = []
    social_links = []

    # Parse domain for filtering internal links
    try:
        domain = urlparse(site_url).netloc
    except:
        return [], [], [], "Invalid URL"

    # Process pages
    page_count = 0
    while queue and page_count < max_pages:
        try:
            current_url = queue.popleft()
            if current_url in visited:
                continue

            # Mark as visited
            visited.add(current_url)
            page_count += 1

            print(f"  Processing page {page_count}/{max_pages}: {current_url}")

            # Extract content
            links, content = extract_links_and_content(current_url)

            # Extract emails
            page_emails = extract_emails(content)
            emails.update(page_emails)

            # Check for job content
            has_job, job_context = detect_job_content(content)
            if has_job:
                job_content.append({
                    "url": current_url,
                    "context": job_context
                })

            # Find social links
            page_social = [link for link in links if any(social in link for social in SOCIAL_PLATFORMS)]
            social_links.extend(page_social)

            # Add internal links to queue
            for link in links:
                full_link = urljoin(current_url, link)
                parsed_link = urlparse(full_link)

                # Only add internal links
                if parsed_link.netloc == domain and full_link not in visited:
                    queue.append(full_link)

            # Add some delay to avoid being blocked
            time.sleep(random.uniform(1, 2))

        except Exception as e:
            print(f"⚠️ Error processing {current_url}: {e}")
            continue

    return list(emails), job_content, list(set(social_links)), "Success"

# ----------------- MAIN EXECUTION -----------------

def process_business(place_id):
    """
    Process a business by its place_id
    """
    # Get place details
    details = get_place_details(place_id)
    if not details:
        return None

    name = details.get("name", "Unknown")
    address = details.get("formatted_address", "Unknown")
    phone = details.get("formatted_phone_number", "Unknown")
    website = details.get("website", "")
    hr_emails = extract_hr_emails(website) if website else ["N/A"]

    print(f"\n📋 Processing: {name}")
    print(f"  📍 Address: {address}")
    print(f"  📞 Phone: {phone}")
    print(f"  🌐 Website: {website}")
    print(f"  🌐 HR Mail: {hr_emails}")

    emails = []
    job_content = []
    social_links = []
    crawl_status = "Not crawled"

    # Crawl website if available
    if website:
        try:
            emails, job_content, social_links, crawl_status = crawl_site(website)
        except Exception as e:
            print(f"⚠️ Error crawling website: {e}")

    # Check Google-provided social links if no website or job content found
    if not website or not job_content:
        google_social = get_social_from_google(details)

        # Add to social links list
        social_links.extend(google_social)

        # Crawl first social link if available and no job content found
        if google_social and not job_content:
            try:
                social_emails, social_job_content, _, _ = crawl_site(google_social[0])
                emails.extend(social_emails)
                job_content.extend(social_job_content)
            except Exception as e:
                print(f"⚠️ Error crawling social link: {e}")

    # Format job content for CSV
    has_job_posting = len(job_content) > 0
    job_urls = [item["url"] for item in job_content]
    job_contexts = [item["context"] for item in job_content]

    result = {
        "name": name,
        "address": address,
        "phone": phone,
        "website": website,
        "emails": "; ".join(emails) if emails else "None found",
        "HR Emails": ",".join(hr_emails),
        "has_job_posting": "Yes" if has_job_posting else "No",
        "job_urls": "; ".join(job_urls) if job_urls else "None found",
        "job_contexts": "; ".join(job_contexts) if job_contexts else "None found",
        "social_links": "; ".join(social_links) if social_links else "None found",
        "crawl_status": crawl_status
    }

    return result

In [ ]:
import re
from selenium.common.exceptions import WebDriverException

def extract_hr_emails(website_url):
    """
    Extract HR-related emails from a given website URL
    """
    try:
        driver = setup_chrome_driver()
        driver.get(website_url)
        page_text = driver.page_source
        driver.quit()

        # Find all email addresses
        all_emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", page_text)
        unique_emails = list(set(all_emails))

        # Filter HR-related emails
        hr_keywords = ['hr', 'people', 'talent', 'careers', 'recruit', 'jobs']
        hr_emails = [email for email in unique_emails if any(kw in email.lower() for kw in hr_keywords)]

        return hr_emails if hr_emails else ["Not Found"]

    except WebDriverException as e:
        print(f"Error accessing {website_url}: {e}")
        return ["Error"]

In [ ]:

def main():
    """
    Main execution function for scraping + saving to Colab filesystem
    """
    all_results = []

    # Ensure output directory exists
    output_dir = "/content/Kobeyo/Scraped CSV"
    os.makedirs(output_dir, exist_ok=True)

    # Process each search tag
    for tag in SEARCH_TAGS:
        place_ids = search_places(tag)

        for place_id in place_ids:
            result = process_business(place_id)
            if result:
                all_results.append(result)
            time.sleep(2)  # Respect API rate limits

    # Save results to CSV
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"{output_dir}/business_job_listings.csv"
        df.to_csv(output_file, index=False)
        print(f"\n✅ Results saved to {output_file}")
        return df
    else:
        print("\n❌ No results found")
        return None

if __name__ == "__main__":
    main()



🔍 Searching for: pet grooming in Los Angeles
✅ Found 20 places
Error accessing https://www.21pooch.com/: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


📋 Processing: 21Pooch Hollywood
  📍 Address: 5137 W Sunset Blvd Unit 213, 2nd floor, Los Angeles, CA 90027, USA
  📞 Phone: (310) 290-3905
  🌐 Website: https://www.21pooch.com/
  🌐 HR Mail: ['Error']

🔍 Crawling: https://www.21pooch.com/
  Processing page 1/5: https://www.21pooch.com/
  Processing page 2/5: https://www.21pooch.com
  Processing page 3/5: https://www.21pooch.com/price
  Processing page 4/5: https://www.21pooch.com/book-now
  Processing page 5/5: https://www.21pooch.com/contacts
Error accessing https://www.21pooch.com/: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_lo